### Purpose
This Jupyter notebook claculates **the concentration of elements (g ELEMENT/L) and (mol ELEMENT/L)** for 4 media used in the experiements DDB_PD_138_AMBR and DDB_PD_139_AMBR:
- DDB_BM_002
- DDB_BM_005  
- DDB_FM_002
- DDB_FM_005

### Excel file: "media_recipe.xlsx"
The file contains the steps followed to get the compound composition for every medium
 
### Excel file: "compounds_database.xlsx"
The file contains the molecular formula for the compound found in the media which will be used to perform all the following calculations


## Loading the files needed for the calculations

In [1]:
from chempy import Substance
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import re
from IPython.display import display

ABSOLUTE_PATH = r"C:/Users/harou/OneDrive/Bureau/data_suresh/"

def searchCompound(search_value):
    index = None
    if search_value.strip():
        for column in df_compounds.columns:
            exact_match = df_compounds[column].str.strip() == search_value.strip()
            if exact_match.any():
                index = exact_match[exact_match].index[0]
                return df_compounds.loc[index]
        return None
    
def getMolarMass(formula):
    return round(Substance.from_formula(formula).mass, 5)


def parseMolecularFormula(molecular_formula, water_count):
    element_counts = {'C': 0, 'H': 0, 'O': 0, 'N': 0, 'S': 0, 'P': 0, 'Cl': 0, 'Na':0, 'Mo':0, 'K': 0, 'Mg': 0, 'Ca': 0, 'Fe': 0, 'Mn': 0, 'Zn': 0, 'Cu': 0, 'Co': 0}

    element_pattern = r'([A-Z][a-z]*)(\d*)'
    matches = re.finditer(element_pattern, molecular_formula)

    for match in matches:
        element_symbol = match.group(1)
        element_count = int(match.group(2)) if match.group(2) else 1
        if element_symbol in element_counts:
            element_counts[element_symbol] += element_count

    # Decreasing the count of H and O based on water_count
    element_counts['H'] -= (2 * water_count)
    element_counts['O'] -= water_count
    
    element_matrix = [[element, count] for element, count in element_counts.items() if count > 0]
    return element_matrix

def sumSimilarElements(element_lists):
    element_counts = {}
    for element_list in element_lists:
        for element, count in element_list:
            if element in element_counts:
                element_counts[element] += count
            else:
                element_counts[element] = count

    return [[element, count] for element, count in element_counts.items()]

def getWaterMoleculesCount(chemical_formula):
    match = re.search(r'\*(\d*)H2O', chemical_formula)
    if match:
        if match.group(1) == '':
            water_molecule_count = 1
        else:
            water_molecule_count = int(match.group(1))
        return water_molecule_count
    else:
        return 0
    
def center_align(val):
    return 'text-align: center'

def printBoldText(text, color=34):
    print(f"\033[1;{color}m{text}\033[0m")
    
pd.set_option('display.max_columns', None)

df_compounds = pd.read_excel(f"{ABSOLUTE_PATH}compounds_database.xlsx", sheet_name="Sheet1")

printBoldText("Loading the registered compounds with their molecular formula:", 30)
display(df_compounds.style.applymap(center_align))

printBoldText("Loading the media composition:", 30)

df_media_composition = pd.read_excel(f"{ABSOLUTE_PATH}media_recipe.xlsx", sheet_name="media_composition", header=None)
media_list = {}

media_list[df_media_composition[0][0]] = df_media_composition[1: 14]
media_list[df_media_composition[0][14]] = df_media_composition[15: 30]
media_list[df_media_composition[0][30]] = df_media_composition[31: 41]
media_list[df_media_composition[0][41]] = df_media_composition[42: 44]

for medium_name, medium_composition in media_list.items():
    printBoldText(f"\n{medium_name}: ", 31)
    medium_composition_copy = medium_composition.copy()
    medium_composition_copy = medium_composition_copy.drop(medium_composition_copy.columns[2], axis=1)
    medium_composition_copy.columns = ["Compound", "Amount (g/L)"]
    medium_composition_copy.reset_index(drop=True, inplace=True)
    display(medium_composition_copy.style.applymap(center_align))
    

Loading the registered compounds with their molecular formula:


,Name,Structure,Molecular formula
0,Glucose monohydrate,C6H12O6*H2O,C6H14O7
1,Sucrose,C12H22O11,C12H22O11
2,Citric acid monohydrate,C6H8O7*H2O,C6H10O8
3,Thiamine,C12H17N4OS,C12H17N4OS
4,Ammonium sulphate,(NH4)2SO4,H8N2O4S
5,Di-ammonium hydrogen phosphate,(NH4)2HPO4,H9N2O4P
6,Ammonium chloride,NH4Cl,H4ClN
7,Monopotassium phosphate,KH2PO4,H2KO4P
8,Dipotassium phosphate,K2HPO4,HK2O4P
9,Potassium chloride,KCl,KCl


Loading the media composition:

DDB_BM_002: 


,Compound,Amount (g/L)
0,C6H12O6*H2O,22.000000
1,KH2PO4,15.700000
2,(NH4)2SO4,7.600000
3,MgSO4*7H2O,0.492940
4,Thiamine,0.100000
5,FeCl3*6H2O,0.041800
6,Na2 EDTA*2H2O,0.033400
7,CaCl2*2H2O,0.002000
8,CoCl2*6H2O,0.000540
9,ZnSO4*7H2O,0.000540



DDB_BM_005: 


,Compound,Amount (g/L)
0,C6H12O6*H2O,22.000000
1,KH2PO4,15.700000
2,(NH4)2SO4,7.600000
3,C6H8O7*H2O,1.860000
4,MgSO4*7H2O,0.492940
5,Thiamine,0.100000
6,FeCl3*6H2O,0.146300
7,Na2 EDTA*2H2O,0.116900
8,FeCl3*6H2O,0.041800
9,ZnSO4*7H2O,0.023490



DDB_FM_002: 


,Compound,Amount (g/L)
0,C6H12O6*H2O,550.000000
1,MgSO4*7H2O,13.700000
2,FeCl3*6H2O,0.104500
3,Na2 EDTA*2H2O,0.083500
4,CaCl2*2H2O,0.005000
5,CoCl2*6H2O,0.001350
6,ZnSO4*7H2O,0.001350
7,Na2MoO4*2H2O,0.001250
8,CuSO4*5H2O,0.001200
9,MnSO4*H2O,0.000750



DDB_FM_005: 


,Compound,Amount (g/L)
0,C6H12O6*H2O,550.000000
1,MgSO4*7H2O,13.700000


# Calculations

In [2]:
df_elements_concentration_list = []

for medium_name, medium_composition in media_list.items():

    medium_compound_compositions = list(media_list[medium_name][0])
    medium_compound_concentartions = list(media_list[medium_name][1])

    elements = ['C', 'H', 'O', 'N', 'S', 'P', 'Cl', 'Na', 'K', 'Mg', 'Ca', 'Fe', 'Mn', 'Zn', 'Cu', 'Co', 'Mo']
    columns = ["Name", "Structure", "MW (g/mol)"] + elements

    df_matrix = pd.DataFrame(columns=columns)

    for index, compound in enumerate(medium_compound_compositions):

        found_compound = searchCompound(compound.strip())
        
        name = found_compound['Name']
        structure = found_compound['Structure']
        molecular_formula = found_compound['Molecular formula']
        water_count = getWaterMoleculesCount(structure)
        
        molecular_weight = getMolarMass(molecular_formula)
        molecular_weight_without_water = getMolarMass(molecular_formula)-water_count*getMolarMass('H2O') # removing the MW of water molecules
        
        # 1. Creating the elemnts matrix 
        elemental_composition = parseMolecularFormula(molecular_formula, water_count) # removing the atoms from the water molecules
        compounds_data = {element: 0 for element in columns}
        for element, count in elemental_composition:
            compounds_data[element] = count

        # 2. Calculating the fraction of element present in the compound (gELEMENT/gCOMPOUND)
        fraction_data = {}
        for element, count in compounds_data.items():
            if element not in ["Name", "Structure", "MW (g/mol)"]:
                fraction_data[f"{element} (g/g)"] = (count * getMolarMass(element) / molecular_weight_without_water)
        compounds_data.update(fraction_data)

        # 3. Calculating the concentration of elements present in the medium (gELEMENT/L)
        concentration_data = {}
        for element, fraction in fraction_data.items():
            # Removing the mass contribution of water
            ajusted_concentration = medium_compound_concentartions[index] * (molecular_weight_without_water/molecular_weight)
            concentration_data[f"{element.replace(' (g/g)', '')} (g/L)"] = fraction*ajusted_concentration
        compounds_data.update(concentration_data)

        # Generating the final dataframe
        compounds_data["Name"] = found_compound['Name']
        compounds_data["Structure"] = found_compound['Structure'].split("*")[0]
        compounds_data["MW (g/mol)"] = molecular_weight_without_water

        df_matrix = df_matrix.append(compounds_data, ignore_index=True)

    column_order = ['Name', 'Structure', 'MW (g/mol)'] + elements + [f"{element} (g/g)" for element in elements] + [f"{element} (g/L)" for element in elements]
    df_matrix = df_matrix[column_order]

    printBoldText(f'\nCalculations for {medium_name}:', 31)
    #printBoldText(f"\n1- The elements composition of each compound \n2- The fraction of elements in each compound (gELEMENT/gCOMPOUND)\n3- The concentration of elements in each compound (gELEMENT/L)", 30)

    display(df_matrix.style.applymap(center_align))

    # 
    printBoldText(f"\nThe concentration of elements in {medium_name} medium  is: ", 34)

    metal_concentration_columns = [col for col in df_matrix.columns if "(g/L)" in col]
    elements_concentration = df_matrix[metal_concentration_columns].sum()

    df_elements_concentration = pd.DataFrame({'Metal ion': [item.replace(' (g/L)', '') for item in metal_concentration_columns], 'g ELEMENT /L': elements_concentration})
    df_elements_concentration.reset_index(drop=True, inplace=True)
    
    #df_elements_concentration = df_elements_concentration.sort_values(by='g ELEMENT /L', ascending=False)
    
    df_elements_concentration.reset_index(drop=True, inplace=True)
    
    df_elements_concentration['mol ELEMENT /L'] = df_elements_concentration.apply(lambda row: row['g ELEMENT /L'] * getMolarMass(row['Metal ion']), axis=1)
       
    df_elements_concentration_list.append({medium_name: df_elements_concentration})
    display(df_elements_concentration.style.applymap(center_align))
    
    
columns = []
double_header = []
data = []

columns.append("Metal ion")
data.append(list(df_elements_concentration_list[0]["DDB_BM_002"]["Metal ion"]))
        
for dictionary in df_elements_concentration_list:
    for medium_name, df in dictionary.items():        
        columns.append(f"{medium_name}" )        
        data.append(list(df["g ELEMENT /L"]))

printBoldText(f"\nSummary: ", 31)
df_summary = pd.DataFrame({columns[0]: data[0], columns[1]: data[1], columns[2]: data[2], columns[3]: data[3], columns[4]: data[4]})

df_summary.columns = pd.MultiIndex.from_tuples([(' ',               columns[0]),
                                                ('g ELEMENT /L',    columns[1]),
                                                ('g ELEMENT /L ',   columns[2]),
                                                ('g ELEMENT /L  ',  columns[3]),
                                                ('g ELEMENT /L   ', columns[4])])

try:
    df_summary.to_excel(f"{ABSOLUTE_PATH}files/summary.xlsx", index=True)
    df_summary_local = pd.read_excel(f"{ABSOLUTE_PATH}files/summary.xlsx", sheet_name="Sheet1")
    df_summary_local = df_summary_local.drop(df_summary_local.index[1])
    df_summary_local = df_summary_local.drop(columns=df_summary_local.columns[0])
    df_summary_local.to_excel(f"{ABSOLUTE_PATH}files/summary.xlsx", index=False)
    
except:
    pass

display(df_summary.style.applymap(center_align))


Calculations for DDB_BM_002:


,Name,Structure,MW (g/mol),C,H,O,N,S,P,Cl,Na,K,Mg,Ca,Fe,Mn,Zn,Cu,Co,Mo,C (g/g),H (g/g),O (g/g),N (g/g),S (g/g),P (g/g),Cl (g/g),Na (g/g),K (g/g),Mg (g/g),Ca (g/g),Fe (g/g),Mn (g/g),Zn (g/g),Cu (g/g),Co (g/g),Mo (g/g),C (g/L),H (g/L),O (g/L),N (g/L),S (g/L),P (g/L),Cl (g/L),Na (g/L),K (g/L),Mg (g/L),Ca (g/L),Fe (g/L),Mn (g/L),Zn (g/L),Cu (g/L),Co (g/L),Mo (g/L)
0,Glucose monohydrate,C6H12O6,180.156000,6,12,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.400020,0.067142,0.532838,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000424,1.342840,10.656796,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Monopotassium phosphate,KH2PO4,136.084060,0,2,4,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.000000,0.014814,0.470268,0.000000,0.000000,0.227608,0.000000,0.000000,0.287310,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.232586,7.383210,0.000000,0.000000,3.573439,0.000000,0.000000,4.510766,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Ammonium sulphate,(NH4)2SO4,132.134000,0,8,4,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.061029,0.484327,0.212012,0.242632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.463820,3.680882,1.611292,1.844007,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Magnesium sulfate heptahydrate,MgSO4,120.361000,0,0,4,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0.000000,0.000000,0.531700,0.000000,0.266365,0.000000,0.000000,0.000000,0.000000,0.201934,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.127994,0.000000,0.064121,0.000000,0.000000,0.000000,0.000000,0.048611,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Thiamine,C12H17N4OS,265.355000,12,17,1,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0.543167,0.064578,0.060293,0.211144,0.120819,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.054317,0.006458,0.006029,0.021114,0.012082,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Ferric chloride hexahydrate,FeCl3,162.195000,0,0,0,0,0,0,3,0,0,0,0,1,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.655692,0.000000,0.000000,0.000000,0.000000,0.344308,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016447,0.000000,0.000000,0.000000,0.000000,0.008637,0.000000,0.000000,0.000000,0.000000,0.000000
6,EDTA disodium dihydrate,Na2 EDTA,338.223540,10,16,8,2,0,0,0,2,0,0,0,0,0,0,0,0,0,0.355120,0.047684,0.378424,0.082827,0.000000,0.000000,0.000000,0.135944,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010719,0.001439,0.011423,0.002500,0.000000,0.000000,0.000000,0.004103,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Calcium chloride dihydrate,CaCl2,110.978000,0,0,0,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.638865,0.000000,0.000000,0.000000,0.361135,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000965,0.000000,0.000000,0.000000,0.000545,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Cobalt chloride hexahydrate,CoCl2,129.833190,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.546085,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.453915,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000134,0.000000
9,Zinc sulfate heptahydrate,ZnSO4,161.436000,0,0,4,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0.000000,0.000000,0.396417,0.000000,0.198593,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

,Metal ion,g ELEMENT /L,mol ELEMENT /L
0,C,8.065460,96.874236
1,H,2.047143,2.063520
2,O,21.866823,349.847301
3,N,1.634906,22.900129
4,S,1.920389,61.567657
5,P,3.573439,110.682829
6,Cl,0.017573,0.622952
7,Na,0.004198,0.096521
8,K,4.510766,176.363271
9,Mg,0.048611,1.181485



The concentration of elements in DDB_BM_002 medium  is: 

Calculations for DDB_BM_005:


,Name,Structure,MW (g/mol),C,H,O,N,S,P,Cl,Na,K,Mg,Ca,Fe,Mn,Zn,Cu,Co,Mo,C (g/g),H (g/g),O (g/g),N (g/g),S (g/g),P (g/g),Cl (g/g),Na (g/g),K (g/g),Mg (g/g),Ca (g/g),Fe (g/g),Mn (g/g),Zn (g/g),Cu (g/g),Co (g/g),Mo (g/g),C (g/L),H (g/L),O (g/L),N (g/L),S (g/L),P (g/L),Cl (g/L),Na (g/L),K (g/L),Mg (g/L),Ca (g/L),Fe (g/L),Mn (g/L),Zn (g/L),Cu (g/L),Co (g/L),Mo (g/L)
0,Glucose monohydrate,C6H12O6,180.156000,6,12,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.400020,0.067142,0.532838,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000424,1.342840,10.656796,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Monopotassium phosphate,KH2PO4,136.084060,0,2,4,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.000000,0.014814,0.470268,0.000000,0.000000,0.227608,0.000000,0.000000,0.287310,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.232586,7.383210,0.000000,0.000000,3.573439,0.000000,0.000000,4.510766,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Ammonium sulphate,(NH4)2SO4,132.134000,0,8,4,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.061029,0.484327,0.212012,0.242632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.463820,3.680882,1.611292,1.844007,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Citric acid monohydrate,C6H8O7,192.123000,6,8,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.375103,0.041973,0.582923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.637880,0.071377,0.991287,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Magnesium sulfate heptahydrate,MgSO4,120.361000,0,0,4,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0.000000,0.000000,0.531700,0.000000,0.266365,0.000000,0.000000,0.000000,0.000000,0.201934,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.127994,0.000000,0.064121,0.000000,0.000000,0.000000,0.000000,0.048611,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Thiamine,C12H17N4OS,265.355000,12,17,1,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0.543167,0.064578,0.060293,0.211144,0.120819,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.054317,0.006458,0.006029,0.021114,0.012082,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Ferric chloride hexahydrate,FeCl3,162.195000,0,0,0,0,0,0,3,0,0,0,0,1,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.655692,0.000000,0.000000,0.000000,0.000000,0.344308,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.057565,0.000000,0.000000,0.000000,0.000000,0.030228,0.000000,0.000000,0.000000,0.000000,0.000000
7,EDTA disodium dihydrate,Na2 EDTA,338.223540,10,16,8,2,0,0,0,2,0,0,0,0,0,0,0,0,0,0.355120,0.047684,0.378424,0.082827,0.000000,0.000000,0.000000,0.135944,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037517,0.005038,0.039979,0.008750,0.000000,0.000000,0.000000,0.014362,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Ferric chloride hexahydrate,FeCl3,162.195000,0,0,0,0,0,0,3,0,0,0,0,1,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.655692,0.000000,0.000000,0.000000,0.000000,0.344308,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016447,0.000000,0.000000,0.000000,0.000000,0.008637,0.000000,0.000000,0.000000,0.000000,0.000000
9,Zinc sulfate heptahydrate,ZnSO4,161.436000,0,0,4,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0.000000,0.000000,0.396417,0.000000,0.198593,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

...

,Name,Structure,MW (g/mol),C,H,O,N,S,P,Cl,Na,K,Mg,Ca,Fe,Mn,Zn,Cu,Co,Mo,C (g/g),H (g/g),O (g/g),N (g/g),S (g/g),P (g/g),Cl (g/g),Na (g/g),K (g/g),Mg (g/g),Ca (g/g),Fe (g/g),Mn (g/g),Zn (g/g),Cu (g/g),Co (g/g),Mo (g/g),C (g/L),H (g/L),O (g/L),N (g/L),S (g/L),P (g/L),Cl (g/L),Na (g/L),K (g/L),Mg (g/L),Ca (g/L),Fe (g/L),Mn (g/L),Zn (g/L),Cu (g/L),Co (g/L),Mo (g/L)
0,Glucose monohydrate,C6H12O6,180.156000,6,12,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.400020,0.067142,0.532838,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,200.010597,33.571007,266.419910,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Magnesium sulfate heptahydrate,MgSO4,120.361000,0,0,4,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0.000000,0.000000,0.531700,0.000000,0.266365,0.000000,0.000000,0.000000,0.000000,0.201934,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.557266,0.000000,1.782079,0.000000,0.000000,0.000000,0.000000,1.351012,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000



The concentration of elements in DDB_FM_005 medium  is: 


,Metal ion,g ELEMENT /L,mol ELEMENT /L
0,C,200.010597,2402.327279
1,H,33.571007,33.839575
2,O,269.977176,4319.364845
3,N,0.000000,0.000000
4,S,1.782079,57.133468
5,P,0.000000,0.000000
6,Cl,0.000000,0.000000
7,Na,0.000000,0.000000
8,K,0.000000,0.000000
9,Mg,1.351012,32.836344



Summary: 


,,g ELEMENT /L,g ELEMENT /L,g ELEMENT /L,g ELEMENT /L
,Metal ion,DDB_BM_002,DDB_BM_005,DDB_FM_002,DDB_FM_005
0,C,8.065460,8.730137,200.037395,200.010597
1,H,2.047143,2.122119,33.574605,33.571007
2,O,21.866823,22.892696,270.006955,269.977176
3,N,1.634906,1.641156,0.006250,0.000000
4,S,1.920389,1.923244,1.782526,1.782079
5,P,3.573439,3.573439,0.000000,0.000000
6,Cl,0.017573,0.077952,0.043932,0.000000
7,Na,0.004198,0.014695,0.010496,0.000000
8,K,4.510766,4.510766,0.000000,0.000000
